In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os.path as op
import torch
import argparse
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
from torch.nn import init
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import random
import time
import math
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import re
IMAGE_WIDTH = {5: 15, 20: 60, 60: 180}
IMAGE_HEIGHT = {5: 32, 20: 64, 60: 96}    

# Check the shape of input data frame

In [2]:
year = 2017

In [3]:
label_df = pd.read_feather(op.join("./img_data/monthly_20d", f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))
label_df.shape
#print((label_df["Retx_20d_label"]==2).sum()) 

(67858, 35)

## Design Data
Read data, data split, and define transformation and noralization

In [4]:
#Read data
class ImageData(Dataset): #Dataset as a base class
    def __init__(self, data_root = './img_data/monthly_20d', split = 'train'):
        self.data_root = data_root
        self.split = split
        self.idx_to_kv = []#list
        self.images = {}#dictionary, key = year
        self.labels = {}
        assert split in ["train","valid","test"]
        #Split testing set
        if split == "test":
            for year in range(2000, 2020):
                label_df = pd.read_feather(op.join(self.data_root, f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))#read features
                n = label_df.shape[0]
                self.images[year] = np.memmap(op.join(self.data_root, f"20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                    (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))#read images
                label_df['pred_label'] = 0
                self.labels[year] = label_df
                for i in range(0,n):
                    if label_df["Retx_20d_label"][i] != 2:
                        self.idx_to_kv += [(year,i)]
        else:
            for year in range(1993, 2000):
                label_df = pd.read_feather(op.join(self.data_root, f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))
                n = label_df.shape[0]
                self.images[year] = np.memmap(op.join(self.data_root, f"20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                    (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20]))
#                 label_df['pred_label'] = 0
                
                self.labels[year] = label_df
                for i in range(0,n):
                    if label_df["Retx_20d_label"][i] != 2:
                        self.idx_to_kv += [(year,i)] 
                assert self.images[year].shape[0] == self.labels[year].shape[0]
            random.seed(2021)
            random.shuffle(self.idx_to_kv)
            
            #split into train and validation samples
            train_len = int(0.7 * len(self.idx_to_kv))
            if split == "train":
                self.idx_to_kv  = self.idx_to_kv[:train_len]
            else:
                self.idx_to_kv = self.idx_to_kv[train_len:]
                
            
        self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),  # this also convert pixel value from [0,255] to [0,1]，as NN prefers decimal
                transforms.Normalize((0.1307,), (0.3081,)),#normalization
            ])#transfer to form of pytorch -- Tensor

    def __len__(self):
         return len(self.idx_to_kv)  

    
    def __getitem__(self, index):
        year, idx = self.idx_to_kv[index]
        label_df = self.labels[year]
        images = self.images[year]
        label = label_df["Retx_20d_label"][idx]
        img = images[idx]
        img = self.transform(img)
        return img, label.astype(np.int_), index #to int64

# a = ImageData(split = "valid")
# print(a.labels)
            

## Architecture Design
With both baseline model and 16 more models(based on Table18 in the paper) for sensitivity analysis

In [14]:
class CNN(nn.Module):#nn.Module as a base class
    def __init__(self,channels: int = 64, 
                 layers: int = 3, 
                 drop_rate: float = 0.5, 
                 use_BN: bool = True,
                 use_Xavier: bool = True,
                 activation: str = "LReLU",
                 pool_size = (2,1),
                 filter_size = (5,3),
                 dilation = (2,1),
                 stride = (3,1)
                 ): #define parameters of baseline model given in the paper, with some variation, to obtain a corret output of size 46080
        
        super(CNN, self).__init__()#nn.Module inistialized
        model = []
        def conv_pool(channels_in, channels_out, filter_size=(5,3),stride=1,dilation=(1,1),pool_size = (2,1),activation = "LReLU"):#channels_in：RGB=3 B&W=1
            padding = (int((dilation[0] * filter_size[0] - 1) / 2), int((dilation[1] * filter_size[1] - 1) / 2))
            layer = []
            layer += [nn.Conv2d(channels_in, channels_out, kernel_size=filter_size,stride=stride,dilation=dilation,padding=padding)]
            if use_BN:
                 layer += [nn.BatchNorm2d(channels_out)]
            if activation == "LReLU":
                layer += [nn.LeakyReLU(True)]
            elif activation == "ReLU":
                layer += [nn.ReLU(True)]
            else:
                raise NotImplementedError("Activation function %s is not supported in CNN"%activation)
            layer += [nn.MaxPool2d(pool_size)]
            return layer
        
        #Calculate the output size for different models
        def cal_output_size(input_size, stride, pool_size):
            h_output = int((int((input_size[0] - 1) / stride[0]) + 1) / pool_size[0])
            w_output = int((int((input_size[1] - 1) / stride[1]) + 1) / pool_size[1])
            return (h_output, w_output)
            
      
        model += conv_pool(1,channels, filter_size = filter_size, stride=stride, \
                           dilation=dilation, pool_size = pool_size, activation = activation)  #valid settings,to get an output of size 46080*2
        output_size = cal_output_size((64,60), stride, pool_size)
        # model += conv_pool_dropout(1,64,stride=(3,1),dilation=(2,1)) #settings in paper
        for i in (0, layers - 1):
            model += conv_pool(channels,channels * 2,filter_size = filter_size, pool_size = pool_size, activation = activation)
            channels *= 2
            output_size = cal_output_size(output_size, (1,1), pool_size)
        # model += conv_pool_dropout(channels,channels * 2)

        
        model += [nn. Flatten(1)]
        
        fc_channels = output_size[0] * output_size[1] * channels
        #add dropout to fully connected layer
        model += [nn.Dropout(drop_rate)]
        model += [nn.Linear(fc_channels,2)]
        self.model = nn.Sequential(*model)
        
        #initialize all functions above(nn.Conv2d, nn.BatchNorm2d, nn.LeakyReLU,nn.MaxPool2d)

        def init_func(m):
            classname = m.__class__.__name__
            if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
                init.xavier_normal_(m.weight.data)
#                 if hasattr(m, 'bias') and m.bias is not None:
#                     init.constant_(m.bias.data, 0.0)
#             elif classname.find('BatchNorm2d') != -1:
#                 init.normal_(m.weight.data, 1.0)
#                 init.constant_(m.bias.data, 0.0)
        if use_Xavier:      
            self.apply(init_func)
        
    def forward(self, input):
        return self.model(input)
    
#Final model
model = CNN()
a = torch.rand(1,1,64,60)
print(model(a).shape)
print(model)

torch.Size([1, 2])
CNN(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 3), stride=(3, 1), padding=(4, 1), dilation=(2, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=True)
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=True)
    (7): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=True)
    (11): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_d

## Define training and testing process (after validation)

In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train() # train mode
    for batch_idx, (imgs, labels, _) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad() #clear gradient of last train
        output = model(imgs) 
#         print(output[0], labels[0])
        loss = F.cross_entropy(output, labels, ignore_index=2) # cross_entropy loss = Logsoftmax and NLLLoss
        
#         pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#         correct = pred.eq(labels.view_as(pred)).sum().item() # calculate correct forecast
#         print(correct)
        loss.backward() 
        optimizer.step() # gradient descent
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(imgs), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break



def test(model, device, test_loader):
    model.eval() # eval mode
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for imgs, labels, indices in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)
#             print(output[0])
            test_loss += F.cross_entropy(output, labels, ignore_index=2, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()
#             if save_labels:
#                 for i, index in enumerate(indices):
#                     year, idx = test_dataset.idx_to_kv[index]
#                     pred_label = pred[i].item()
#                     test_dataset.labels[year]['pred_label'][idx] = pred_label

    test_loss /= len(test_loader.dataset) # avg the loss in test, compare it with train loss
    accuracy = 100. * correct / len(test_loader.dataset) #calculate accuracy
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    return test_loss, accuracy

def hyper_para():
    parser = argparse.ArgumentParser(description='PyTorch IPT')
    args = parser.parse_args(args=[])
    
    args.batch_size = 1280      # input batch size for training (default: 1280) 128 in paper, change to 1280 to speed the training process
    args.test_batch_size = 3000 # input batch size for testing (default: 1280) 128 in paper, change to 1280 to speed the training process
    args.epochs = 14            # number of epochs to train (default: 14) Not mentioned in paper
    args.lr = 0.0001            # learning rate (default: 0.0001) 
    args.gamma = 0.9            # Learning rate step gamma (default: 0.9) Not mentioned in paper
    args.no_cuda = False        # disables CUDA training 
    args.dry_run = False        # quickly check a single pass
    args.seed = 2021            # random seed (default: 2021)
    args.log_interval = 10      # how many batches to wait before logging training status
    return args
  

In [7]:
# Training settings
args = hyper_para()
  
use_cuda = not args.no_cuda and torch.cuda.is_available() # use GPU:YES

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu") 

train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 8,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)
    test_kwargs['shuffle'] = False

train_dataset = ImageData(split = "train")
valid_dataset = ImageData(split = "valid")
test_dataset = ImageData(split = "test")
train_loader = DataLoader(train_dataset,**train_kwargs) 
valid_loader = DataLoader(valid_dataset, **test_kwargs) 
test_loader = DataLoader(test_dataset, **test_kwargs) 

## Run all the models & save with paras and results in the file names accordingly

In [21]:
def run(model_kwargs):  
    model = CNN(**model_kwargs).to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr) # learning rate
    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma) # adjust learning rate: lr * gamma
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        scheduler.step(
        
    valid_loss, valid_accuracy = test(model, device, valid_loader) # valid loader
    test_loss, test_accuracy = test(model, device, test_loader) # test loader
    
    
    model_name = "cnn_for_20d"
    model_kwargs['val_loss'] = '%.4f'%valid_loss
    model_kwargs['val_acc'] = '%.3f'%valid_accuracy
    model_kwargs['test_loss'] = '%.4f'%test_loss
    model_kwargs['test_acc'] = '%.3f'%test_accuracy
    for key, value in model_kwargs.items():
        model_name = model_name + "_%s=%s"%(key, value) 
    torch.save(model.state_dict(), model_name + '.pt')

List of models( baseline and variation)

In [66]:
model_kwargs_list = [
    {},
    {'channels': 32},
    {'channels': 128},
    {'layers': 2},
    {'layers': 4},
    {'drop_rate': 0.},
    {'drop_rate': 0.25},
    {'drop_rate': 0.75},
    {'use_BN': False},
    {'use_Xavier': False},
    {'activation': 'ReLU'},
    {'pool_size': (2,2)},
    {'filter_size': (3,3)},
    {'filter_size': (7,3)},
    {'dilation': (2,1),'stride': (1,1)},
    {'dilation': (1,1),'stride': (3,1)},
    {'dilation': (1,1),'stride': (1,1)}   
]


Run (Note that the training process is not shown here fully as models have been saved in root)

In [10]:
for model_kwargs in model_kwargs_list:
    run(model_kwargs)

Train Epoch: 1 [0/482925 (0%)]	Loss: 1.243759
Train Epoch: 1 [12800/482925 (3%)]	Loss: 1.152234
Train Epoch: 1 [25600/482925 (5%)]	Loss: 1.231294
Train Epoch: 1 [38400/482925 (8%)]	Loss: 1.156597
Train Epoch: 1 [51200/482925 (11%)]	Loss: 1.126944
Train Epoch: 1 [64000/482925 (13%)]	Loss: 1.113608
Train Epoch: 1 [76800/482925 (16%)]	Loss: 1.113961
Train Epoch: 1 [89600/482925 (19%)]	Loss: 1.019400
Train Epoch: 1 [102400/482925 (21%)]	Loss: 1.079407
Train Epoch: 1 [115200/482925 (24%)]	Loss: 1.035876
Train Epoch: 1 [128000/482925 (26%)]	Loss: 1.071396
Train Epoch: 1 [140800/482925 (29%)]	Loss: 1.004523
Train Epoch: 1 [153600/482925 (32%)]	Loss: 0.978449
Train Epoch: 1 [166400/482925 (34%)]	Loss: 1.014956
Train Epoch: 1 [179200/482925 (37%)]	Loss: 1.022197
Train Epoch: 1 [192000/482925 (40%)]	Loss: 0.982402
Train Epoch: 1 [204800/482925 (42%)]	Loss: 1.016832
Train Epoch: 1 [217600/482925 (45%)]	Loss: 0.998763
Train Epoch: 1 [230400/482925 (48%)]	Loss: 0.948071
Train Epoch: 1 [243200/48292

Train Epoch: 5 [102400/482925 (21%)]	Loss: 0.696378
Train Epoch: 5 [115200/482925 (24%)]	Loss: 0.696874
Train Epoch: 5 [128000/482925 (26%)]	Loss: 0.719778
Train Epoch: 5 [140800/482925 (29%)]	Loss: 0.692762
Train Epoch: 5 [153600/482925 (32%)]	Loss: 0.703698
Train Epoch: 5 [166400/482925 (34%)]	Loss: 0.695617
Train Epoch: 5 [179200/482925 (37%)]	Loss: 0.685971
Train Epoch: 5 [192000/482925 (40%)]	Loss: 0.700173
Train Epoch: 5 [204800/482925 (42%)]	Loss: 0.704877
Train Epoch: 5 [217600/482925 (45%)]	Loss: 0.710667
Train Epoch: 5 [230400/482925 (48%)]	Loss: 0.701172
Train Epoch: 5 [243200/482925 (50%)]	Loss: 0.724441
Train Epoch: 5 [256000/482925 (53%)]	Loss: 0.709758
Train Epoch: 5 [268800/482925 (56%)]	Loss: 0.708105
Train Epoch: 5 [281600/482925 (58%)]	Loss: 0.703268
Train Epoch: 5 [294400/482925 (61%)]	Loss: 0.717064
Train Epoch: 5 [307200/482925 (63%)]	Loss: 0.701254
Train Epoch: 5 [320000/482925 (66%)]	Loss: 0.704052
Train Epoch: 5 [332800/482925 (69%)]	Loss: 0.708083
Train Epoch:

Train Epoch: 9 [192000/482925 (40%)]	Loss: 0.688953
Train Epoch: 9 [204800/482925 (42%)]	Loss: 0.700333
Train Epoch: 9 [217600/482925 (45%)]	Loss: 0.698455
Train Epoch: 9 [230400/482925 (48%)]	Loss: 0.675985
Train Epoch: 9 [243200/482925 (50%)]	Loss: 0.690149
Train Epoch: 9 [256000/482925 (53%)]	Loss: 0.698523
Train Epoch: 9 [268800/482925 (56%)]	Loss: 0.688923
Train Epoch: 9 [281600/482925 (58%)]	Loss: 0.678948
Train Epoch: 9 [294400/482925 (61%)]	Loss: 0.685350
Train Epoch: 9 [307200/482925 (63%)]	Loss: 0.688667
Train Epoch: 9 [320000/482925 (66%)]	Loss: 0.688195
Train Epoch: 9 [332800/482925 (69%)]	Loss: 0.687249
Train Epoch: 9 [345600/482925 (71%)]	Loss: 0.680986
Train Epoch: 9 [358400/482925 (74%)]	Loss: 0.690080
Train Epoch: 9 [371200/482925 (77%)]	Loss: 0.684094
Train Epoch: 9 [384000/482925 (79%)]	Loss: 0.687211
Train Epoch: 9 [396800/482925 (82%)]	Loss: 0.691427
Train Epoch: 9 [409600/482925 (85%)]	Loss: 0.686631
Train Epoch: 9 [422400/482925 (87%)]	Loss: 0.688611
Train Epoch:

Train Epoch: 13 [256000/482925 (53%)]	Loss: 0.693044
Train Epoch: 13 [268800/482925 (56%)]	Loss: 0.678765
Train Epoch: 13 [281600/482925 (58%)]	Loss: 0.675802
Train Epoch: 13 [294400/482925 (61%)]	Loss: 0.678434
Train Epoch: 13 [307200/482925 (63%)]	Loss: 0.683014
Train Epoch: 13 [320000/482925 (66%)]	Loss: 0.685679
Train Epoch: 13 [332800/482925 (69%)]	Loss: 0.692418
Train Epoch: 13 [345600/482925 (71%)]	Loss: 0.675286
Train Epoch: 13 [358400/482925 (74%)]	Loss: 0.686406
Train Epoch: 13 [371200/482925 (77%)]	Loss: 0.689397
Train Epoch: 13 [384000/482925 (79%)]	Loss: 0.684605
Train Epoch: 13 [396800/482925 (82%)]	Loss: 0.680033
Train Epoch: 13 [409600/482925 (85%)]	Loss: 0.682266
Train Epoch: 13 [422400/482925 (87%)]	Loss: 0.683858
Train Epoch: 13 [435200/482925 (90%)]	Loss: 0.684958
Train Epoch: 13 [448000/482925 (93%)]	Loss: 0.674669
Train Epoch: 13 [460800/482925 (95%)]	Loss: 0.684228
Train Epoch: 13 [473600/482925 (98%)]	Loss: 0.692086
Train Epoch: 14 [0/482925 (0%)]	Loss: 0.680092

Train Epoch: 3 [307200/482925 (63%)]	Loss: 0.713373
Train Epoch: 3 [320000/482925 (66%)]	Loss: 0.722034
Train Epoch: 3 [332800/482925 (69%)]	Loss: 0.734462
Train Epoch: 3 [345600/482925 (71%)]	Loss: 0.725310
Train Epoch: 3 [358400/482925 (74%)]	Loss: 0.717858
Train Epoch: 3 [371200/482925 (77%)]	Loss: 0.722500
Train Epoch: 3 [384000/482925 (79%)]	Loss: 0.725312
Train Epoch: 3 [396800/482925 (82%)]	Loss: 0.726194
Train Epoch: 3 [409600/482925 (85%)]	Loss: 0.722770
Train Epoch: 3 [422400/482925 (87%)]	Loss: 0.711824
Train Epoch: 3 [435200/482925 (90%)]	Loss: 0.701989
Train Epoch: 3 [448000/482925 (93%)]	Loss: 0.719750
Train Epoch: 3 [460800/482925 (95%)]	Loss: 0.718172
Train Epoch: 3 [473600/482925 (98%)]	Loss: 0.734817
Train Epoch: 4 [0/482925 (0%)]	Loss: 0.704120
Train Epoch: 4 [12800/482925 (3%)]	Loss: 0.718475
Train Epoch: 4 [25600/482925 (5%)]	Loss: 0.715750
Train Epoch: 4 [38400/482925 (8%)]	Loss: 0.712229
Train Epoch: 4 [51200/482925 (11%)]	Loss: 0.713783
Train Epoch: 4 [64000/482

...

Train Epoch: 10 [307200/482925 (63%)]	Loss: 0.688844
Train Epoch: 10 [320000/482925 (66%)]	Loss: 0.693183
Train Epoch: 10 [332800/482925 (69%)]	Loss: 0.685681
Train Epoch: 10 [345600/482925 (71%)]	Loss: 0.686562
Train Epoch: 10 [358400/482925 (74%)]	Loss: 0.687064
Train Epoch: 10 [371200/482925 (77%)]	Loss: 0.680460
Train Epoch: 10 [384000/482925 (79%)]	Loss: 0.693339
Train Epoch: 10 [396800/482925 (82%)]	Loss: 0.686577
Train Epoch: 10 [409600/482925 (85%)]	Loss: 0.680411
Train Epoch: 10 [422400/482925 (87%)]	Loss: 0.694260
Train Epoch: 10 [435200/482925 (90%)]	Loss: 0.688687
Train Epoch: 10 [448000/482925 (93%)]	Loss: 0.709947
Train Epoch: 10 [460800/482925 (95%)]	Loss: 0.694924
Train Epoch: 10 [473600/482925 (98%)]	Loss: 0.695162
Train Epoch: 11 [0/482925 (0%)]	Loss: 0.681537
Train Epoch: 11 [12800/482925 (3%)]	Loss: 0.678358
Train Epoch: 11 [25600/482925 (5%)]	Loss: 0.682908
Train Epoch: 11 [38400/482925 (8%)]	Loss: 0.686383
Train Epoch: 11 [51200/482925 (11%)]	Loss: 0.673629
Train 

Train Epoch: 14 [358400/482925 (74%)]	Loss: 0.679236
Train Epoch: 14 [371200/482925 (77%)]	Loss: 0.675686
Train Epoch: 14 [384000/482925 (79%)]	Loss: 0.678885
Train Epoch: 14 [396800/482925 (82%)]	Loss: 0.666760
Train Epoch: 14 [409600/482925 (85%)]	Loss: 0.692153
Train Epoch: 14 [422400/482925 (87%)]	Loss: 0.690747
Train Epoch: 14 [435200/482925 (90%)]	Loss: 0.682134
Train Epoch: 14 [448000/482925 (93%)]	Loss: 0.675204
Train Epoch: 14 [460800/482925 (95%)]	Loss: 0.672213
Train Epoch: 14 [473600/482925 (98%)]	Loss: 0.676247

Test set: Average loss: 0.6906, Accuracy: 111052/206969 (53.656%)


Test set: Average loss: 0.6967, Accuracy: 775967/1491887 (52.012%)

Train Epoch: 1 [0/482925 (0%)]	Loss: 1.273118
Train Epoch: 1 [12800/482925 (3%)]	Loss: 1.232955
Train Epoch: 1 [25600/482925 (5%)]	Loss: 1.276612
Train Epoch: 1 [38400/482925 (8%)]	Loss: 1.150184
Train Epoch: 1 [51200/482925 (11%)]	Loss: 1.096598
Train Epoch: 1 [64000/482925 (13%)]	Loss: 1.134127
Train Epoch: 1 [76800/482925 (16%)]

Train Epoch: 4 [409600/482925 (85%)]	Loss: 0.779023
Train Epoch: 4 [422400/482925 (87%)]	Loss: 0.792149
Train Epoch: 4 [435200/482925 (90%)]	Loss: 0.774970
Train Epoch: 4 [448000/482925 (93%)]	Loss: 0.760019
Train Epoch: 4 [460800/482925 (95%)]	Loss: 0.749298
Train Epoch: 4 [473600/482925 (98%)]	Loss: 0.764977
Train Epoch: 5 [0/482925 (0%)]	Loss: 0.794463
Train Epoch: 5 [12800/482925 (3%)]	Loss: 0.745561
Train Epoch: 5 [25600/482925 (5%)]	Loss: 0.757174
Train Epoch: 5 [38400/482925 (8%)]	Loss: 0.770023
Train Epoch: 5 [51200/482925 (11%)]	Loss: 0.741329
Train Epoch: 5 [64000/482925 (13%)]	Loss: 0.783101
Train Epoch: 5 [76800/482925 (16%)]	Loss: 0.732150
Train Epoch: 5 [89600/482925 (19%)]	Loss: 0.766001
Train Epoch: 5 [102400/482925 (21%)]	Loss: 0.745737
Train Epoch: 5 [115200/482925 (24%)]	Loss: 0.736504
Train Epoch: 5 [128000/482925 (26%)]	Loss: 0.771749
Train Epoch: 5 [140800/482925 (29%)]	Loss: 0.743101
Train Epoch: 5 [153600/482925 (32%)]	Loss: 0.750979
Train Epoch: 5 [166400/48292

Train Epoch: 9 [12800/482925 (3%)]	Loss: 0.726367
Train Epoch: 9 [25600/482925 (5%)]	Loss: 0.713248
Train Epoch: 9 [38400/482925 (8%)]	Loss: 0.710893
Train Epoch: 9 [51200/482925 (11%)]	Loss: 0.720704
Train Epoch: 9 [64000/482925 (13%)]	Loss: 0.703120
Train Epoch: 9 [76800/482925 (16%)]	Loss: 0.705645
Train Epoch: 9 [89600/482925 (19%)]	Loss: 0.704584
Train Epoch: 9 [102400/482925 (21%)]	Loss: 0.718055
Train Epoch: 9 [115200/482925 (24%)]	Loss: 0.714739
Train Epoch: 9 [128000/482925 (26%)]	Loss: 0.689018
Train Epoch: 9 [140800/482925 (29%)]	Loss: 0.708529
Train Epoch: 9 [153600/482925 (32%)]	Loss: 0.718236
Train Epoch: 9 [166400/482925 (34%)]	Loss: 0.703590
Train Epoch: 9 [179200/482925 (37%)]	Loss: 0.712530
Train Epoch: 9 [192000/482925 (40%)]	Loss: 0.697843
Train Epoch: 9 [204800/482925 (42%)]	Loss: 0.700736
Train Epoch: 9 [217600/482925 (45%)]	Loss: 0.704936
Train Epoch: 9 [230400/482925 (48%)]	Loss: 0.704415
Train Epoch: 9 [243200/482925 (50%)]	Loss: 0.717434
Train Epoch: 9 [256000

Train Epoch: 13 [76800/482925 (16%)]	Loss: 0.695998
Train Epoch: 13 [89600/482925 (19%)]	Loss: 0.689834
Train Epoch: 13 [102400/482925 (21%)]	Loss: 0.696176
Train Epoch: 13 [115200/482925 (24%)]	Loss: 0.694068
Train Epoch: 13 [128000/482925 (26%)]	Loss: 0.699858
Train Epoch: 13 [140800/482925 (29%)]	Loss: 0.691621
Train Epoch: 13 [153600/482925 (32%)]	Loss: 0.686130
Train Epoch: 13 [166400/482925 (34%)]	Loss: 0.690345
Train Epoch: 13 [179200/482925 (37%)]	Loss: 0.696361
Train Epoch: 13 [192000/482925 (40%)]	Loss: 0.687384
Train Epoch: 13 [204800/482925 (42%)]	Loss: 0.680766
Train Epoch: 13 [217600/482925 (45%)]	Loss: 0.677506
Train Epoch: 13 [230400/482925 (48%)]	Loss: 0.701134
Train Epoch: 13 [243200/482925 (50%)]	Loss: 0.681553
Train Epoch: 13 [256000/482925 (53%)]	Loss: 0.695466
Train Epoch: 13 [268800/482925 (56%)]	Loss: 0.691239
Train Epoch: 13 [281600/482925 (58%)]	Loss: 0.690709
Train Epoch: 13 [294400/482925 (61%)]	Loss: 0.713863
Train Epoch: 13 [307200/482925 (63%)]	Loss: 0.68

## Sensitivity Analysis
After saving models accordingly, we show the results and clculate the correlations and Sharpe ratios to make comparison.

In [67]:
import os 
file_list = os.listdir(".")
model_name_list = ['baseline']
model_file_list = [next(file_name for file_name in file_list if file_name.startswith('cnn_for_20d_v'))]
model_kwargs_to_file = [{'model_kwargs':{}, "model_file_name":\
                          next(file_name for file_name in file_list if file_name.startswith('cnn_for_20d_v'))}]
for model_kwargs in model_kwargs_list[1:]:
    model_name = 'cnn_for_20d'
    for key, value in model_kwargs.items():
        model_name = model_name + "_%s=%s"%(key, value) 
    model_file_name = next(file_name for file_name in file_list if file_name.startswith(model_name))
    model_file_list += [model_file_name]
    model_name_list += [model_name[12:]]
    model_kwargs_to_file += [{'model_kwargs':model_kwargs, "model_file_name":model_file_name}]
print(model_kwargs_to_file)

[{'model_kwargs': {}, 'model_file_name': 'cnn_for_20d_val_loss=0.6899_val_acc=54.095_test_loss=0.6978_test_acc=51.898.pt'}, {'model_kwargs': {'channels': 32}, 'model_file_name': 'cnn_for_20d_channels=32_val_loss=0.6876_val_acc=54.032_test_loss=0.6935_test_acc=52.029.pt'}, {'model_kwargs': {'channels': 128}, 'model_file_name': 'cnn_for_20d_channels=128_val_loss=0.6990_val_acc=53.399_test_loss=0.7088_test_acc=51.454.pt'}, {'model_kwargs': {'layers': 2}, 'model_file_name': 'cnn_for_20d_layers=2_val_loss=0.6894_val_acc=53.974_test_loss=0.6956_test_acc=52.201.pt'}, {'model_kwargs': {'layers': 4}, 'model_file_name': 'cnn_for_20d_layers=4_val_loss=0.6915_val_acc=53.592_test_loss=0.6974_test_acc=52.153.pt'}, {'model_kwargs': {'drop_rate': 0.0}, 'model_file_name': 'cnn_for_20d_drop_rate=0.0_val_loss=0.7192_val_acc=53.128_test_loss=0.7328_test_acc=50.920.pt'}, {'model_kwargs': {'drop_rate': 0.25}, 'model_file_name': 'cnn_for_20d_drop_rate=0.25_val_loss=0.6949_val_acc=53.808_test_loss=0.7044_test

## Calculate correlations and Sharpe ratios

In [68]:
results = pd.DataFrame(columns=['Validation Loss','Validation Accuracy','Test Loss','Test Accuracy'\
                               ,'Spearman Correlation','Pearson Correlation','Annual Sharpe Ratio EW rf=0'\
                               ,'Annual Sharpe Ratio EW rf=0.01',],index= model_name_list)
pred_retx_20d_all = pd.DataFrame(columns=['20-day predicted return'])

for model_kwarg_to_file in model_kwargs_to_file:
    print(model_kwarg_to_file)
    model = CNN(**model_kwarg_to_file['model_kwargs']).to(device)
    model.load_state_dict(torch.load(model_kwarg_to_file['model_file_name']))#### read models saved
    model.eval()

    def test2(model, device, test_loader, test_dataset):
        model.eval() # eval mode
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for imgs, labels, indices in test_loader:  
                imgs, labels = imgs.to(device), labels.to(device)
                output = model(imgs)

                test_loss += F.cross_entropy(output, labels, ignore_index=2, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(labels.view_as(pred)).sum().item()
            
                for i, index in enumerate(indices):
                    year, idx = test_dataset.idx_to_kv[index]
                    pred_label = pred[i].item()
                    test_dataset.labels[year].loc[idx,'pred_label'] = pred_label
    
    test2(model, device, test_loader, test_dataset)
    
    
    years = list(range(2000, 2020))
    Sharpe = pd.DataFrame(columns=['Sharpe Ratio'],index=years)
    cal_corr = pd.DataFrame(columns=['Retx_20d_label','pred_label'])

    for year in range(2000, 2020):
        cal_data = test_dataset.labels[year][['Date','StockID','Retx_20d','Retx_20d_label','pred_label']]
        cal_data.insert(cal_data.shape[1],'w',0)

        #correlation
        cal_corr = pd.concat([cal_corr,cal_data[['Retx_20d_label','pred_label']]])

        #Sharpe ratio, equally weighted
        Dates = cal_data['Date'].unique() 
        pred_retx_20d = pd.DataFrame(columns=['20-day predicted return'],index=Dates)

        for date in Dates:
            same_date = cal_data.loc[cal_data['Date'] == date]
            n = len(same_date)

            w_u = 1/n
            w_d = -1/n

            up = (same_date['pred_label']== 1)
            down = (same_date['pred_label']== 0)
            same_date.loc[up, 'w'] = w_u
            same_date.loc[down, 'w'] = w_d
            pred_retx_20d.loc[date,'20-day predicted return'] = (same_date['w'] * same_date['Retx_20d']).sum()
        
        pred_retx_20d_all = pd.concat([pred_retx_20d_all, pred_retx_20d])
    
    #correlations
    #Spearman
    corr_Spearman, _ = spearmanr(cal_corr['Retx_20d_label'],cal_corr['pred_label'])
    #Pearson
    corr_Pearson, _ = pearsonr(cal_corr['Retx_20d_label'],cal_corr['pred_label'])
    #Sharpe ratio, equally weighted
    rf0 = 0
    rf1 = 0.01
    n = 252/20
    sigma = pred_retx_20d_all.std()
    exp_20d_return = pred_retx_20d_all.mean()
    Sharpe_rf0 = ((exp_20d_return-(rf0/n))* math.sqrt(n)/sigma).values 
    Sharpe_rf1 = ((exp_20d_return-(rf1/n))* math.sqrt(n)/sigma).values

    find  = r'cnn_for_20d_(.*?)_val_loss'
    model_name  = re.findall(find, model_kwarg_to_file['model_file_name']) 
    if model_name == []:
        model_name = 'baseline'
    
    b_val_loss = model_kwarg_to_file['model_file_name'].index('val_loss=')+9
    b_val_acc = model_kwarg_to_file['model_file_name'].index('val_acc=')+8
    b_test_loss = model_kwarg_to_file['model_file_name'].index('test_loss=')+10
    b_test_acc = model_kwarg_to_file['model_file_name'].index('test_acc=')+9

    results.loc[model_name,'Validation Loss'] = float(model_kwarg_to_file['model_file_name'][b_val_loss:b_val_loss+6])
    results.loc[model_name,'Validation Accuracy'] = float(model_kwarg_to_file['model_file_name'][b_val_acc:b_val_acc+6])
    results.loc[model_name,'Test Loss'] = float(model_kwarg_to_file['model_file_name'][b_test_loss:b_test_loss+6])
    results.loc[model_name,'Test Accuracy'] = float(model_kwarg_to_file['model_file_name'][b_test_acc:b_test_acc+6])
    results.loc[model_name,'Spearman Correlation'] = format(corr_Spearman,'.4f')
    results.loc[model_name,'Pearson Correlation'] = format(corr_Pearson,'.4f')
    results.loc[model_name,'Annual Sharpe Ratio EW rf=0'] = format(float(Sharpe_rf0),'.4f')
    results.loc[model_name,'Annual Sharpe Ratio EW rf=0.01'] = format(float(Sharpe_rf1),'.4f')
    

{'model_kwargs': {}, 'model_file_name': 'cnn_for_20d_val_loss=0.6899_val_acc=54.095_test_loss=0.6978_test_acc=51.898.pt'}
{'model_kwargs': {'channels': 32}, 'model_file_name': 'cnn_for_20d_channels=32_val_loss=0.6876_val_acc=54.032_test_loss=0.6935_test_acc=52.029.pt'}
{'model_kwargs': {'channels': 128}, 'model_file_name': 'cnn_for_20d_channels=128_val_loss=0.6990_val_acc=53.399_test_loss=0.7088_test_acc=51.454.pt'}
{'model_kwargs': {'layers': 2}, 'model_file_name': 'cnn_for_20d_layers=2_val_loss=0.6894_val_acc=53.974_test_loss=0.6956_test_acc=52.201.pt'}
{'model_kwargs': {'layers': 4}, 'model_file_name': 'cnn_for_20d_layers=4_val_loss=0.6915_val_acc=53.592_test_loss=0.6974_test_acc=52.153.pt'}
{'model_kwargs': {'drop_rate': 0.0}, 'model_file_name': 'cnn_for_20d_drop_rate=0.0_val_loss=0.7192_val_acc=53.128_test_loss=0.7328_test_acc=50.920.pt'}
{'model_kwargs': {'drop_rate': 0.25}, 'model_file_name': 'cnn_for_20d_drop_rate=0.25_val_loss=0.6949_val_acc=53.808_test_loss=0.7044_test_acc=51

Show the results

In [69]:
results

,Validation Loss,Validation Accuracy,Test Loss,Test Accuracy,Spearman Correlation,Pearson Correlation,Annual Sharpe Ratio EW rf=0,Annual Sharpe Ratio EW rf=0.01
baseline,0.6899,54.095,0.6978,51.898,0.0286,0.0228,0.8081,0.6366
channels=32,0.6876,54.032,0.6935,52.029,0.0281,0.0216,0.9647,0.7738
channels=128,0.699,53.399,0.7088,51.454,0.0158,0.0093,1.0099,0.7907
layers=2,0.6894,53.974,0.6956,52.201,0.0269,0.0192,1.1293,0.9127
layers=4,0.6915,53.592,0.6974,52.153,0.0247,0.0165,1.1945,0.9826
drop_rate=0.0,0.7192,53.128,0.7328,50.92,0.0158,0.0113,0.8529,0.6749
drop_rate=0.25,0.6949,53.808,0.7044,51.526,0.0209,0.0151,0.8515,0.6689
drop_rate=0.75,0.6885,53.861,0.6947,51.651,0.0329,0.0283,0.7229,0.5596
use_BN=False,0.6888,54.187,0.6975,51.724,0.0260,0.0203,0.7245,0.5596
use_Xavier=False,0.6935,53.697,0.7016,51.841,0.0194,0.0117,0.7678,0.5988
